Method for loading training data that uses the webdatasets library and saves us a ton of disk and ram issues.

In [ ]:
!pip install webdataset

In [ ]:
import nibabel as nb
import numpy as np
from io import BytesIO
from nibabel import FileHolder, Nifti1Image
import os
import torch
from skimage import transform
import webdataset as wds

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
dataset = wds.Dataset("./drive/MyDrive/macai_datasets/brats_training.tar.gz")

In [ ]:
import nibabel as nb
import numpy as np
from io import BytesIO
from nibabel import FileHolder, Nifti1Image
import os
import torch
from skimage import transform
import webdataset as wds


train_dataset = wds.Dataset("./drive/MyDrive/macai_datasets/brats_training.tar.gz")
eval_dataset = wds.Dataset("./drive/MyDrive/macai_datasets/brats_training.tar.gz")



def col_img(batch):
    bytes_data_list = [list(batch[i].items())[1][1] for i in range(5)]                    
              
    bb = BytesIO(bytes_data_list[0])
    fh = FileHolder(fileobj=bb)
    f_flair = Nifti1Image.from_file_map({'header': fh, 'image': fh}).get_fdata()

    bb = BytesIO(bytes_data_list[1])
    fh = FileHolder(fileobj=bb)
    f_seg = Nifti1Image.from_file_map({'header': fh, 'image': fh}).get_fdata()

    bb = BytesIO(bytes_data_list[2])
    fh = FileHolder(fileobj=bb)
    f_t1 = Nifti1Image.from_file_map({'header': fh, 'image': fh}).get_fdata()

    bb = BytesIO(bytes_data_list[3])
    fh = FileHolder(fileobj=bb)
    f_t1ce = Nifti1Image.from_file_map({'header': fh, 'image': fh}).get_fdata()  

    bb = BytesIO(bytes_data_list[4])
    fh = FileHolder(fileobj=bb)
    f_t2 = Nifti1Image.from_file_map({'header': fh, 'image': fh}).get_fdata()  

    f_t1 = transform.resize(f_t1, [320, 400, 320])
    f_t2 = transform.resize(f_t2, [320, 400, 320])
    f_t1ce = transform.resize(f_t1ce, [320, 400, 320])
    f_flair = transform.resize(f_flair, [320, 400, 320])
    f_seg = transform.resize(f_seg, [320, 400, 320])
    return [torch.tensor(np.stack([f_t1, f_t1ce, f_t2, f_flair])), torch.tensor(f_seg)]

In [ ]:
#Very janky way of seperating into train and validation 
eval_dataset.select(lambda x : int(list(x.items())[0][1].split('/')[-1][17]) == 3)
train_dataset.select(lambda x : int(list(x.items())[0][1].split('/')[-1][17]) < 3)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=5,collate_fn=col_img)
eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=5,collate_fn=col_img)